In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import random
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  for gpu in gpus:
    try:
      print("Enabling GPU: ", gpu)
      tf.config.experimental.set_visible_devices(gpu, 'GPU')
      tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
      print(e)

Enabling GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
val_size = 0.3
rand_seed = random.randint(0, 2**32 - 1)
img_dim = 96
batch_size = 8

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "training",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "validation",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

Found 1970 files belonging to 2 classes.
Using 1379 files for training.
Found 1970 files belonging to 2 classes.
Using 591 files for validation.


2025-02-09 08:43:17.689324: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-02-09 08:43:17.689349: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-02-09 08:43:17.689356: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
I0000 00:00:1739108597.689372 51377810 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1739108597.689394 51377810 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
image_input = keras.Input(shape = (img_dim, img_dim, 3))
x = keras.layers.Rescaling(1.0 / 255)(image_input)
x = keras.layers.Conv2D(128, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
feature_output = keras.layers.Dropout(0.05)(x)

x = keras.layers.Flatten()(feature_output)
x = keras.layers.Dense(2048, activation = 'relu')(x)
classification_output = keras.layers.Dense(2)(x)

model = keras.models.Model(image_input, classification_output)

In [6]:
opt = keras.optimizers.Adagrad(learning_rate = 0.01)
model.compile(optimizer = opt,
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 94, 94, 128)    │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 47, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 123904)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │   253,757,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 254,060,290 (969.16 MB)

 Trainable params: 254,060,290 (969.16 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
class_names = train_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE
epochs = 16

In [8]:
hist = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs, 
  callbacks = []
)

Epoch 1/16


2025-02-09 08:43:18.141103: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


173/173 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - accuracy: 0.6580 - loss: 0.7208 - val_accuracy: 0.8968 - val_loss: 0.2692
Epoch 2/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 12s 72ms/step - accuracy: 0.9380 - loss: 0.1833 - val_accuracy: 0.9594 - val_loss: 0.1153
Epoch 3/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.9652 - loss: 0.0794 - val_accuracy: 0.9662 - val_loss: 0.0871
Epoch 4/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.9819 - loss: 0.0544 - val_accuracy: 0.9695 - val_loss: 0.0932
Epoch 5/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 74ms/step - accuracy: 0.9913 - loss: 0.0322 - val_accuracy: 0.9712 - val_loss: 0.0844
Epoch 6/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 74ms/step - accuracy: 0.9931 - loss: 0.0250 - val_accuracy: 0.9763 - val_loss: 0.0798
Epoch 7/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.9915 - loss: 0.0214 - val_accuracy: 0.9746 - val_loss: 0.0806
Epoch 8/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 76ms/step - accuracy: 0.9954 - loss: 0.0131 - val_accurac

In [9]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "test"),
  shuffle = False,
  image_size = (img_dim, img_dim)
)

Found 30 files belonging to 2 classes.


In [10]:
model.evaluate(test_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0062


[0.0062363059259951115, 1.0]